In [ ]:
# Gravitational Lensing Research
# Program to take a qlens-generated sample chain, derive kappa vs. radius values. No plotting.
# v4: Simultaneously calculate all matter, dark matter only, and baryons.
# Switching to kpc rather than arc seconds.
# Also calculate the average dark matter density within 25 kpc and 50 kpc (rho_25 and rho_50)
# Also calculate the radius containing 1% of the M200 mass
# This version is for MS2137.

In [ ]:
import time
import matplotlib.pyplot as plt
import sys
import numpy as np
import os
import timeit
from IPython.core.debugger import set_trace
import datetime
import concurrent.futures as cf
import multiprocessing as mp
import copy
from scipy import interpolate
from scipy.optimize import minimize_scalar
import pandas as pd
from astropy.cosmology import FlatLambdaCDM
import astropy.units as u
from astropy import constants as const

In [ ]:
# for parallel processing
nproc = 1

##  Changing the output radii values to kpc. Make sure redshift is correct.

In [ ]:
# Names:    A611, A2537, RXCJ2248, MS2137, A383, A2261, M2129, A1703,  M1720
# Reshifts:  0.288, 0.294, 0.348, 0.314, 0.189, 0.225, 0.589, 0.280, 0.387

In [ ]:
zlens = 0.314

In [ ]:
name = 'MS2137'
suffix = '.cnfw.V13'
label = name + suffix
# Set your path here, to the "chains_..." directory of interest
#path = '/home/grant/Documents/qlens-beta/' + name + '/chains_' + label
# path = '/home/kea/KEVIN/UCI/Research/gravlensing/' + name + '/chains_' + label
path = '/Users/kevin/CloudStation/KEVIN/UCI/Research/gravlensing/' + name + '/chains_' + label

In [ ]:

cosmo = FlatLambdaCDM(H0=70 * u.km / u.s /u.Mpc, Om0=0.3)
print(cosmo)
rho_crit = cosmo.critical_density(zlens).to(u.M_sun / u.kpc**3)
print(rho_crit)
kpc_per_arcsec = cosmo.kpc_proper_per_arcmin(zlens) * u.arcmin / (60. * u.arcsec)
print (kpc_per_arcsec)
rho_200 = rho_crit * 200 * kpc_per_arcsec**3  # units of M_sun arcsec^-3
print('rho_200 = ', rho_200)
# get rid of units now
kpc_per_arcsec = kpc_per_arcsec.value
rho_200 = rho_200.value

In [ ]:
# Calculate Sigma_crit, assuming a source redshift of 2.0.
sigma_crit = (const.c**2 * cosmo.angular_diameter_distance(2.0) / (4. * np.pi * const.G *\
                           cosmo.angular_diameter_distance(zlens) * cosmo.angular_diameter_distance_z1z2(zlens, 2.0))\
             ).to(u.M_sun / u.kpc**2)
print(sigma_crit)   
# get rid of units now
sigma_crit = sigma_crit.value

In [ ]:
# radii values, in KPC, log spaced. Don't change.
rstart = 2.5
rstop  = 397.28
nr = 40
radii_kpc = np.logspace(np.log10(rstart), np.log10(rstop), nr, endpoint=False)
radii_as = [np.round(r / kpc_per_arcsec, 2) for r in radii_kpc]
radii_kpc = np.round(radii_kpc, 2)

print(f'Covers radii of {radii_as[0]} arcsec to {radii_as[-1]} arcsec.')

In [ ]:
# now calculated the median and +/- 1-sigma values, accounting for sample weights.
def find_bands(p, samples, n_rad):
    # 'p' is the offset of the first column to sort by.
    # 'n_rad' is the number of columns to sort.
    # The first column is the weight.
    total = [np.sum(samples[:, i + p] * samples[:, 0]) for i in range(n_rad)]
    sigma_minus  = np.zeros(n_rad)
    sigma_plus = np.zeros(n_rad)
    median = np.zeros(n_rad)
    for i in range(n_rad):
        runsum = 0.
        sm_flag = sp_flag = m_flag = False
        # sort the table by increasing kappa for the radius in question
        samsort =  samples[samples[:, i + p].argsort()]
        for row in range(len(samsort)):
            runsum += (samsort[row, 0] * samsort[row, i + p]) / total[i]
            if (not sm_flag) and (runsum > 0.158):
                sm_flag = True
                sigma_minus[i] = samsort[row, i + p]
            if (not sp_flag) and (runsum > 0.841):
                sp_flag = True
                sigma_plus[i] = samsort[row, i + p] 
            if (not m_flag) and (runsum > 0.5):
                m_flag = True
                median[i] = samsort[row, i + p]
    return sigma_minus, sigma_plus, median

In [ ]:
def ql_script4(i, p, params, tstart, ns):
    # assembles and runs the QLens script, returning three vectors of kappa values
    # Note that the first and pth columns are not parameters.
    # the pth column is the chi square.
    #like,  mvir, c, rc_kpc, q, theta, xc, yc, shear1, shear2, mtot1, mtot2, raw_chisq, chisq = params[0:p] 
    
    like, mvir, c, rc_kpc, q, theta, xc, yc, shear1, shear2, mtot1, mtot2, raw_chisq, chisq = params[0:p]
    
    pars0 = ' '.join([str(item) for item in [mvir, c, rc_kpc, q, theta, xc, yc]]) # Halo parameters
    
    pars1 = ' '.join([str(item) for item in [shear1, shear2]]) # Shear parameters

    pars2 = ' '.join([str(item) for item in [mtot1, 15.0, 0.3, 0.92, -17.5, 0, 0]]) # BCG parameters

    pars3 = ' '.join([str(item) for item in [mtot2, 3.84, 0.3, 0.465, -52.12, -9.2, 14.0]]) # GR parameters

    kappaname = 'plotdata/kappadist_temp_' + str(mp.current_process().name) + '.tmp'
    kappaname_dmo = 'plotdata/kappadist_temp_' + str(mp.current_process().name) + '.dmo.tmp'

    script_part_a1 = '''
    # skeleton script to load bestfit results
    zlens=0.314    # REVIEW  (and don't change the "# REVIEW" comment!)
    zsrc=2.0
    zsrc_ref=2.0
    shear_components on
    major_axis_along_y off    

    sci_notation off

    # gridtype cartesian      # REVIEW
    # grid -70 65 -45 55      # REVIEW
    imgdata read ../MS2137_SL_Data_V4.dat   #REVIEW
    '''

    script_part_a2 = '''
    lens clear
    lens cnfw pmode=3 '''
    # here we insert the parameters for the first lens (halo)

    script_part_b1 =''' shear='''
    # here we insert the paramters for the shear
    
    script_part_b2 ='''
    lens pjaffe pmode=2 '''
    # here we insert the parameters for the second lens (BCG)

    script_part_b3 ='''
    lens pjaffe pmode=2 '''
    # here we insert the params for anchor galaxy, assuming there is one. If not, comment out "lens pjaffe pmode=2"

    script_part_c = '''
    # no other perturbers for this one. 
    '''
    # Other perturbers, if any, can go in part c above.
    
    script_part_d = '''
    plotkappa ''' + str(rstart / kpc_per_arcsec) + ' ' + str(rstop / kpc_per_arcsec)\
    + ' ' + str(nr) + ' '

    script_part_e = '''
    cosmology
    '''
    
    script = script_part_a1 + script_part_a2 + pars0 + script_part_b1 + pars1\
        + script_part_b2 + pars2 + script_part_b3 + pars3  \
        + script_part_c + script_part_d + kappaname + ' \n' \
        + script_part_d + kappaname_dmo + ' lens=0\n' \
        + script_part_e
    sc_name = 'plotdata/kappascript_' + str(mp.current_process().name) + '.in'
    
    # double check redshift
    rs_pos = script.find('zlens=')
    if rs_pos == -1:
        print("Can't locate 'zlens=' in script.")
        sys.exit()    
    eol = script.find('# REVIEW', rs_pos)
    assert float(script[rs_pos + 6: eol]) == zlens
    
    with open(sc_name, 'w') as outfile:
        outfile.write(script)

    # Now, run the script, saving the output in a temporary file
    os.system('qlens ' + sc_name + ' -q | tee plotdata/scriptout' + str(mp.current_process().name) + '.tmp')
    
    # read back the cosmology and determine the r_200 of lens 0
    with open('plotdata/scriptout' + str(mp.current_process().name) + '.tmp', 'r') as outf:
        output = ''.join(outf.readlines())
    cos_pos = output.find('cosmology')
    lens0_pos = output.find('Lens 0:', cos_pos)
    r200_pos = output.find('r_200 = ', lens0_pos)
    kpc_pos = output.find('kpc', r200_pos)
    r200 = np.float(output[r200_pos + 7 : kpc_pos])
    m200 = 4. * np.pi / 3. * r200**3 * rho_200 / kpc_per_arcsec**3 
    
    # read in kappa file back in, and store results
    kdat = np.loadtxt(kappaname)
    kappa = kdat[:, 1]
    kdat_dmo = np.loadtxt(kappaname_dmo)
    kappa_dmo = kdat_dmo[:, 1]
    encl_mass = kdat_dmo[:, 4]
    # The above assumes that the QLens data is in M_sun.
    kappa_bary = kappa - kappa_dmo
    
    # Calculate the radius containing 1% of m200:
    f_encl = interpolate.interp1d(radii_kpc, encl_mass, fill_value='extrapolate')
    r_1_res = minimize_scalar(lambda r: np.abs(f_encl(r) - 0.01 * m200))
    if r_1_res.success != True:
        set_trace()
    r_1pct = r_1_res.x
    if r_1pct <0. or r_1pct > 4000.:
        set_trace()
    
    if i==0:
        assert np.all(np.abs((kdat[:, 0] * kpc_per_arcsec - radii_kpc) / radii_kpc) \
                      < .01)
    if i%50 == 1:
        tavg = (timeit.default_timer() - tstart) / (i+1.)
        proj_fin =  tavg * (ns -i -1)
        sys.stdout.write("\r{0:5.2%} complete. Avg time per iteration: {1:8.6f} sec. Projected finish in {2:4.2f} min. "\
                   .format((float(i) / ns), tavg,  proj_fin/60))
    sys.stdout.flush() # required in multiprocessing or else it buffers this output
    return kappa, kappa_dmo, kappa_bary, encl_mass, r_1pct

In [ ]:
# Now doing this calculation not in a function.

os.chdir(path)
print(os.getcwd())
# Import the chain. This is the weigthed chain.
samples = np.loadtxt(path + '/' + label, comments="#", delimiter=None, unpack=False)
with open(path + '/' + label + '.paramnames') as afile:
    paramnames = afile.readlines()
paramnames = [item[:-1] for item in paramnames]  # Strip off the \n character.
samples = np.array(samples)
# prune out samples with very small weights.
mask = np.where(samples[:, 0] > 1e-20)
samples = samples[mask]
ns, p = samples.shape
print(samples.shape)
print(paramnames)

chain_columns = ['weight'] + paramnames + ['loglike']

# for testing only:
# ns = 500
# samples = samples[-ns:]

In [ ]:
# make sure there is a directory called plotdata. If not, create it.
if not os.path.exists(os.getcwd()+'/plotdata'):
    os.makedirs(os.getcwd()+'/plotdata')

results = []
tstart = timeit.default_timer()
with cf.ProcessPoolExecutor(nproc) as pp:
    for i, params in enumerate(samples[:, 0:p]):
        results.append((i, pp.submit(ql_script4, i, p, params, tstart, ns)))

# code waits here until all processes are finished
sys.stdout.write('\r                                                                                      ')
print("\nKappa calculation completed in ", np.round((timeit.default_timer() - tstart) / 60, 2), " minutes.")

In [ ]:
all_names = ['all_%07.2f' % rad for rad in radii_kpc]
dm_names = ['dm_%07.2f' % rad for rad in radii_kpc]
bary_names = ['bary_%07.2f' % rad for rad in radii_kpc]
m_names = ['m_encl_%07.2f' % rad for rad in radii_kpc]
col_names = all_names + dm_names + bary_names + m_names + ['r_1pct']

# Use a numpy array for holding everything. DataFrames are too slow.
samples2 = np.append(samples, np.full((ns, nr * 4 + 1), np.nan), axis=1)

old_i = -1
print('Updating data array...')
for (i,k) in results:
    k_all, k_dm, k_bary, m_encl, r_1= k.result()
    reslist = list(k_all) + list(k_dm) + list(k_bary) + list(m_encl)+ [r_1]
    samples2[i, p:] = reslist
    if old_i +1 != i:
        print('WARNING: Out of order at i=', i)
    old_i = i
print('Data updated.')

print('Making bands...')
sm, sp, med = find_bands(1, np.hstack((samples2[:, 0:1], samples2[:, p: p + nr])), nr)
sm_dmo, sp_dmo, med_dmo = find_bands(1, np.hstack((samples2[:, 0:1], samples2[:, p + nr: p + nr * 2])), nr)
sm_bary, sp_bary, med_bary = find_bands(1, np.hstack((samples2[:, 0:1], samples2[:, p + nr * 2: p + nr *3])), nr)
sm_mencl, sp_mencl, med_mencl = find_bands(1, np.hstack((samples2[:, 0:1], samples2[:, p + nr * 3: p + nr * 4])), nr)
sm_r1, sp_r1, med_r1 = find_bands(1, np.hstack((samples2[:, 0:1], samples2[:, p + nr * 4: p + nr * 4 + 1])), 1)
print("Bands made.")

In [ ]:
# Multiply the kappas in the samples2 array by Sigma_crit, to yield surface density in M_sun kpc^-2
samples2[:, p: p + nr * 3] = samples2[:, p: p + nr * 3] * sigma_crit
df = pd.DataFrame(samples2, columns = chain_columns + col_names)
df.describe()

In [ ]:
np.savez(label + '.all.bands', radii_kpc, sm, sp, med)
np.savez(label + '.dmo.bands', radii_kpc, sm_dmo, sp_dmo, med_dmo)
np.savez(label + '.bary.bands', radii_kpc, sm_bary, sp_bary, med_bary)
np.savez(label + '.mencl.bands', radii_kpc, sm_mencl, sp_mencl, med_mencl)
np.savez(label + '.r1pct.bands', radii_kpc, sm_r1, sp_r1, med_r1)
with open(label + '.column_names', 'w') as colfile:
    colfile.writelines('\n'.join(list(df.columns.values)))
df.to_csv(label + '.csv')
print('Results saved to bands file, and dataframe saved to .csv file with label ', label)
print('Path: ', path)

In [ ]:
# Use Notebook "Kappa band plotter v4" to plot the results.